<a href="https://colab.research.google.com/github/shangeth/wavencoder/blob/master/examples/notebooks/wavencoder_demo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Notebook
## [wavencoder](https://pypi.org/project/wavencoder/) Models

---


Author : Shangeth Rajaa

![Twitter Follow](https://img.shields.io/twitter/follow/shangethr?style=social)

[GitHub](https://github.com/shangeth) [LinkedIn](https://www.linkedin.com/in/shangeth/)

# Installing wavencoder

In [ ]:
!pip install fairseq
!pip install wavencoder

     |████████████████████████████████| 307kB 4.7MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2046423 sha256=f68a8ad6a5ce93540273cc72dd2d81ba74683c60a9aeb5aad1b93c187a2bb6e5
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


# Wav2Vec pretrained feature extractor

In [1]:
import torch
import wavencoder

x = torch.randn(1, 16000) # [1, 16000]
encoder = wavencoder.models.Wav2Vec(pretrained=False)
z = encoder(x) # [1, 512, 99]
z.shape

torch.Size([1, 512, 98])

# SincNet pretrained feature extractor

In [3]:
from wavencoder.models import SincNet
encoder = SincNet(pretrained=False).eval()
x = torch.randn(1, 3200) 
z = encoder(x)
print(z.shape)

torch.Size([1, 2048])


# RawNet

In [3]:
import torch
import wavencoder

x = torch.randn(1, 59049) # [1, 16000]
encoder = wavencoder.models.RawNet2Model(pretrained_path='/home/shangeth/Downloads/rawnet2_best_weights.pt', return_code=True, class_dim=100)
z = encoder(x) # [1, 512, 99]
z.shape

torch.Size([1, 1024])

In [9]:
import fairseq
fairseq.__version__

'0.10.2'

In [4]:
encoder

RawNet2Model(
  (encoder): RawNet2(
    (ln): LayerNorm()
    (first_conv): SincConv_fast()
    (first_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lrelu): LeakyReLU(negative_slope=0.01)
    (lrelu_keras): LeakyReLU(negative_slope=0.3)
    (block0): Sequential(
      (0): Residual_block_wFRM(
        (lrelu): LeakyReLU(negative_slope=0.01)
        (lrelu_keras): LeakyReLU(negative_slope=0.3)
        (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (mp): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
        (frm): FRM(
          (fc): Linear(in_features=128, out_features=128, bias=True)
          (sig): Sigmoid()
        )
      )
    )
    (block1): Sequential(
      (0): Residual_block_wFRM(
        (bn1): Bat

In [6]:
path = '/home/shangeth/Downloads/rawnet2_best_weights.pt'
pre = torch.load(path)
# vars().keys()
pre

OrderedDict([('ln.gamma',
              tensor([ 1.2650e-05,  5.8814e-05, -1.4774e-04,  ...,  6.3899e-06,
                       1.2560e-39,  1.2560e-39], device='cuda:0')),
             ('ln.beta',
              tensor([-3.1122e-04, -2.0728e-04,  1.4851e-04,  ...,  7.0294e-05,
                       0.0000e+00,  0.0000e+00], device='cuda:0')),
             ('first_conv.low_hz_',
              tensor([[6.1414e+00],
                      [1.5379e+01],
                      [2.6749e+01],
                      [3.9432e+01],
                      [5.2995e+01],
                      [6.7355e+01],
                      [8.2085e+01],
                      [9.7288e+01],
                      [1.1291e+02],
                      [1.2836e+02],
                      [1.4493e+02],
                      [1.6188e+02],
                      [1.7919e+02],
                      [1.9687e+02],
                      [2.1493e+02],
                      [2.3335e+02],
                      [2.5108e+02],
     

# Audio Classifier
- wav2vec encoder `[1, 16000] -> [1, 512, 98]`
- mean of features along time axis `[1, 512, 98] -> [1, 512]`
- ANN Classifier `[1, 512] -> [1, 2]`


In [5]:
import torch
import torch.nn as nn
import wavencoder

class AudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = wavencoder.models.Wav2Vec(pretrained=False)
        self.classifier = nn.Linear(512, 2)

    def forward(self, x):
        z = self.encoder(x)
        z = torch.mean(z, dim=2)
        out = self.classifier(z)
        return out

model = AudioClassifier()
x = torch.randn(1, 16000)
y_hat = model(x)
print(y_hat.shape)

torch.Size([1, 2])


- SincNet encoder `[1, 3200] -> [1, 6420]`
- ANN Classifier `[1, 6420] -> [1, 512]`

In [8]:
import torch
import torch.nn as nn
import wavencoder

class SincNetAudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = SincNet(pretrained=False)
        self.classifier = nn.Linear(2048, 2)

    def forward(self, x):
        z = self.encoder(x)
        out = self.classifier(z)
        return out

model = SincNetAudioClassifier()
x = torch.randn(2, 3200)
y_hat = model(x)
print(y_hat.shape)

torch.Size([2, 2])


# LSTM Attention Classifier

In [5]:
import torch
import torch.nn as nn
import wavencoder

model = nn.Sequential(
        wavencoder.models.Wav2Vec(pretrained=False),
        wavencoder.models.LSTM_Attn_Classifier(512, 64, 2, return_attn_weights=True, attn_type='soft')
)

x = torch.randn(5, 16000)
y_hat, attn_weights = model(x)

print(y_hat.shape, attn_weights.shape)

torch.Size([5, 2]) torch.Size([5, 98])
